In [ ]:
# import libraries
from cil.optimisation.algorithms import PDHG
from cil.optimisation.functions import L2NormSquared, MixedL21Norm, BlockFunction, IndicatorBox, TotalVariation, ZeroFunction
from cil.optimisation.operators import BlockOperator, GradientOperator, IdentityOperator, SymmetrisedGradientOperator, ZeroOperator
from cil.framework import ImageGeometry, AcquisitionGeometry, AcquisitionData, BlockDataContainer
from cil.plugins.astra.operators import ProjectionOperator
from cil.utilities.display import show2D

import numpy as np
import os, sys

import tomophantom
from tomophantom import TomoP2D

In [ ]:
# Load a tomophantom image 
model = 2 # select a model number from the library
N = 128 # set dimension of the phantom
path = os.path.dirname(tomophantom.__file__)
path_library2D = os.path.join(path, "Phantom2DLibrary.dat")

phantom2D_np = TomoP2D.Model(model, N, path_library2D)    
ig = ImageGeometry(voxel_num_x=N, voxel_num_y=N, voxel_size_x = 0.5, voxel_size_y = 0.5)

phantom2D = ig.allocate()
phantom2D.fill(phantom2D_np)

In [ ]:
# Create Acquisition Geometry

detectors =  int(np.sqrt(2)*N)
angles = np.linspace(0, np.pi, 180, dtype=np.float32)

ag = AcquisitionGeometry.create_Parallel2D()\
                        .set_angles(angles,angle_unit="radian")\
                        .set_panel(detectors, pixel_size=0.5)

In [ ]:
device = "gpu"

In [ ]:
# Create Projection Operator
A = ProjectionOperator(ig, ag, device )

In [ ]:
# Create projection data and corrupt with noise
np.random.seed(10)

data = A.direct(phantom2D)
noisy_data = ag.allocate()
noisy_data.fill(data.as_array() + np.random.normal(0, 1, ag.shape))

In [ ]:
# Regularisation parameters
alpha_tgv = 4
beta = 0.5*alpha_tgv

In [ ]:
K11 = GradientOperator(ig)
K12 = IdentityOperator(K11.range)
K22 = SymmetrisedGradientOperator(K11.range)    
K21 = ZeroOperator(ig, K22.range) 
K31 = A
K32 = ZeroOperator(K11.range, ag) 

K = BlockOperator(K11, -K12, K21, K22, K31, K32, shape=(3,2) )   

f1 = alpha_tgv * MixedL21Norm()
f2 = beta * MixedL21Norm() 
f3 = 0.5 * L2NormSquared(b=noisy_data)

F = BlockFunction(f1, f2, f3)         
G = BlockFunction(IndicatorBox(lower=0.0), ZeroFunction())

# Compute operator Norm
normK = K.norm()

# Primal & dual stepsizes
sigma = 1./normK
tau = 1./normK

# Setup and run the PDHG algorithm
pdhg = PDHG(f=F,g=G,operator=K, tau=tau, sigma=sigma,
            max_iteration = 5000,
            update_objective_interval = 500)
pdhg.run(verbose=2)


In [ ]:
show2D([phantom2D, pdhg.solution.get_item(0), pdhg.solution.get_item(1).pnorm(2)],
          title=["Phantom", "PDHG - solution u","PDHG - solution w"],
         origin = "upper", cmap="inferno", num_cols=3)